In [14]:
import pandas as pd
import pandapower as pp
import random
import numpy as np
import time
from utils import run_time_series  # Ensure this is adapted to handle DataFrame inputs directly

# Load CSV data for loads and generations
load_profiles = pd.read_csv("LoadData_C.csv", index_col=0)
load_profiles.columns.name = "household"
generation_profiles = pd.read_csv("GenerationData_C.csv", index_col=0)
generation_profiles.columns.name = "household"

# Define the network creation function
def create_network():
    net = pp.create_empty_network()
    n = 17  # number of buses, loads and generators

    buses = [pp.create_bus(net, vn_kv=20, name="Bus 0")]
    # append a list of 10 buses to the list buses
    buses.extend([pp.create_bus(net, vn_kv=0.4, name=f"Bus {i}") for i in range(1, n + 1)])
    buses.extend([pp.create_bus(net, vn_kv=0.4, name=f"Bus {i}") for i in range(n + 1, n + 1 + 10)])

    nextline = int(np.ceil(n / 2))
    lines1 = [
        pp.create_line(
            net,
            from_bus=buses[i],
            to_bus=buses[i + 1],
            length_km=0.021,
            std_type="NAYY 4x50 SE",
            name=f"Line {i}",
        )
        for i in range(1, int(np.floor(n / 2)))
    ]

    pp.create_line(
        net,
        from_bus=buses[1],
        to_bus=buses[nextline],
        length_km=0.021,
        std_type="NAYY 4x50 SE",
        name=f"connection line",
    )

    lines2 = [
        pp.create_line(
            net,
            from_bus=buses[i],
            to_bus=buses[i + 1],
            length_km=0.021,
            std_type="NAYY 4x50 SE",
            name=f"Line {i}",
        )
        for i in range(int(np.ceil(n / 2)), n)
    ]

    pp.create_line(
        net,
        from_bus=buses[1],
        to_bus=buses[n + 1],
        length_km=0.021,
        std_type="NAYY 4x50 SE",
        name=f"connection line",
    )

    lines3 = [
        pp.create_line(
            net,
            from_bus=buses[i],
            to_bus=buses[i + 1],
            length_km=0.021,
            std_type="NAYY 4x50 SE",
            name=f"Line {i}",
        )
        for i in range(n + 1, n + 10)
    ]

    pp.create_ext_grid(net, buses[0])
    pp.create_transformer(
        net, hv_bus=buses[0], lv_bus=buses[1], std_type="0.4 MVA 20/0.4 kV", name="Trafo"
    )

    # create loads
    loads = [
        pp.create_load(net, buses[i], p_mw=0.0010, q_mvar=0.001, name=f"Load {i}")
        for i in range(1, n + 1 + 10)
    ]
    # create generators
    generators = [
        pp.create_sgen(net, buses[i], p_mw=0.0010, q_mvar=0.001, vm_pu=1.0, name=f"Gen {i}")
        for i in range(1, n + 1 + 10)
    ]
    return net

# Create the network
net = create_network()

# Assigning load_data and gen_data globally for use in the fitness function
load_data = load_profiles
gen_data = generation_profiles

# Define the fitness function
def calculate_total_line_loading(load_order, gen_order):
    num_loads = len(load_data.columns)
    num_gens = len(gen_data.columns)

    # Ensure correct lengths
    assert len(load_order) == num_loads, f"Length mismatch: {len(load_order)} != {num_loads}"
    assert len(gen_order) == num_gens, f"Length mismatch: {len(gen_order)} != {num_gens}"

    # Run the time series simulation
    res_ext, res_lines = run_time_series(
        gen_data,
        load_data,
        net,
        index_order_load=load_order,
        index_order_gen=gen_order,
    )

    # Calculate total line loading and check constraints
    total_line_loading = res_lines.max(axis=1).sum()
    all_within_limits = all(res_lines.max(axis=1) <= 100)

    return total_line_loading if all_within_limits else float('inf')

# Generate initial solution
def generate_initial_solution():
    middle_load_indices = random.sample([i for i in range(27) if i not in [9, 26]], 25)
    load_indices = [9] + middle_load_indices + [26]

    middle_gen_indices = random.sample([i for i in range(27) if i not in [13, 23]], 25)
    gen_indices = [13] + middle_gen_indices + [23]

    return load_indices, gen_indices

# Generate neighboring solution
def generate_neighbor(load_order, gen_order):
    # Swap one pair of elements in the middle part of load_order
    neighbor_load = load_order[:]
    middle_load_indices = list(range(1, len(load_order) - 1))
    idx1, idx2 = random.sample(middle_load_indices, 2)
    neighbor_load[idx1], neighbor_load[idx2] = neighbor_load[idx2], neighbor_load[idx1]

    # Swap one pair of elements in the middle part of gen_order
    neighbor_gen = gen_order[:]
    middle_gen_indices = list(range(1, len(gen_order) - 1))
    idx1, idx2 = random.sample(middle_gen_indices, 2)
    neighbor_gen[idx1], neighbor_gen[idx2] = neighbor_gen[idx2], neighbor_gen[idx1]

    return neighbor_load, neighbor_gen

# Acceptance probability function
def acceptance_probability(old_cost, new_cost, temperature):
    if new_cost < old_cost:
        return 1.0
    else:
        return np.exp((old_cost - new_cost) / temperature)

# Temperature schedule function
def temperature_schedule(t, alpha=0.95):
    return t * alpha

# Simulated Annealing Algorithm with smarter swapping logic
def simulated_annealing(initial_load, initial_gen, initial_temp, alpha, max_iter):
    current_load = initial_load
    current_gen = initial_gen
    current_cost = calculate_total_line_loading(current_load, current_gen)

    best_load = current_load
    best_gen = current_gen
    best_cost = current_cost

    temperature = initial_temp

    for iteration in range(max_iter):
        neighbor_load, neighbor_gen = generate_neighbor(current_load, current_gen)
        neighbor_cost = calculate_total_line_loading(neighbor_load, neighbor_gen)

        if acceptance_probability(current_cost, neighbor_cost, temperature) > random.random():
            current_load = neighbor_load
            current_gen = neighbor_gen
            current_cost = neighbor_cost

        if current_cost < best_cost:
            best_load = current_load
            best_gen = current_gen
            best_cost = current_cost

        temperature = temperature_schedule(temperature, alpha)

        print(f"Iteration {iteration + 1}/{max_iter}, Current Cost: {current_cost}, Best Cost: {best_cost}, Temperature: {temperature}")
        print(f"Load Order: {current_load}")
        print(f"Gen Order: {current_gen}")

    return best_load, best_gen, best_cost

# Function to attempt multiple combinations and analyze positive modifications
def optimize_with_multiple_attempts(num_attempts, initial_temp, alpha, max_iter):
    best_solution = None
    best_cost = float('inf')

    for attempt in range(num_attempts):
        print(f"\nAttempt {attempt + 1}/{num_attempts}:")

        # Generate initial solution for this attempt
        initial_load, initial_gen = generate_initial_solution()

        # Run simulated annealing
        current_load, current_gen, current_cost = simulated_annealing(initial_load, initial_gen, initial_temp, alpha, max_iter)

        # Update best solution if found a better one in this attempt
        if current_cost < best_cost:
            best_solution = (current_load, current_gen)
            best_cost = current_cost

    return best_solution, best_cost

# Set parameters for optimization
num_attempts = 3
initial_temp = 1000
alpha = 0.95
max_iter = 5

# Measure computational time
start_time = time.time()

# Run optimization with multiple attempts
best_solution, best_cost = optimize_with_multiple_attempts(num_attempts, initial_temp, alpha, max_iter)

# Calculate the total computational time
end_time = time.time()
computational_time = end_time - start_time

print(f"\nTotal Computational Time for Simulated Annealing: {computational_time:.2f} seconds")

# Print best solution
best_load, best_gen = best_solution
print("\nBest Total Line Loading:", best_cost)
print("Best Order Load:", best_load)
print("Best Order Gen:", best_gen)

# Save the best order to CSV for evaluation
Group_Letter = 'Z'  # change this to your group letter
df_save = pd.DataFrame({'generation_order': best_gen, 'load_order': best_load})
df_save.to_csv(f'Group_{Group_Letter}_Assignment_2_Data_Set_C_order.csv', index=False)

# Load again to check if it is correct
df = pd.read_csv(f'Group_{Group_Letter}_Assignment_2_Data_Set_C_order.csv')
your_gen_order_loaded = df['generation_order'].values
your_load_order_loaded = df['load_order'].values

# Ensure the orders match
assert (list(best_gen) == list(your_gen_order_loaded)), "Generation order mismatch!"
assert (list(best_load) == list(your_load_order_loaded)), "Load order mismatch!"

# Re-create network to ensure clean state for evaluation
net = create_network()

# Evaluate the results
res_ext, res_lines = run_time_series(gen_data, load_data, net, index_order_load=best_load, index_order_gen=best_gen, results_suffix="evaluation")
assert all(res_lines.max(axis=1) <= 100.0), "Line loading exceeds limit!"
print("Total Line Loading for Evaluation:", res_lines.max(axis=1).sum())



Attempt 1/3:


No time steps to calculate are specified. I'll check the datasource of the first controller for avaiable time steps
100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 24/24 [00:00<00:00, 80.61it/s]
No time steps to calculate are specified. I'll check the datasource of the first controller for avaiable time steps
100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 24/24 [00:00<00:00, 62.98it/s]


Iteration 1/5, Current Cost: 121.4130720399801, Best Cost: 120.78585964217777, Temperature: 950.0
Load Order: [9, 23, 18, 0, 19, 4, 1, 17, 8, 3, 21, 5, 13, 14, 15, 11, 20, 12, 2, 10, 22, 7, 6, 16, 25, 24, 26]
Gen Order: [13, 15, 5, 22, 6, 26, 4, 21, 10, 3, 25, 12, 19, 16, 18, 11, 9, 20, 2, 24, 17, 1, 8, 7, 0, 14, 23]


No time steps to calculate are specified. I'll check the datasource of the first controller for avaiable time steps
100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 24/24 [00:00<00:00, 55.84it/s]


Iteration 2/5, Current Cost: 121.69736484328371, Best Cost: 120.78585964217777, Temperature: 902.5
Load Order: [9, 23, 18, 0, 19, 4, 1, 17, 8, 3, 21, 20, 13, 14, 15, 11, 5, 12, 2, 10, 22, 7, 6, 16, 25, 24, 26]
Gen Order: [13, 15, 5, 22, 6, 26, 4, 21, 10, 3, 25, 12, 19, 16, 18, 11, 9, 20, 2, 24, 17, 1, 0, 7, 8, 14, 23]


...

Iteration 4/5, Current Cost: 111.55046447063715, Best Cost: 111.55046447063715, Temperature: 814.5062499999999
Load Order: [9, 10, 16, 6, 1, 7, 22, 14, 23, 20, 21, 17, 0, 3, 12, 25, 5, 19, 4, 13, 8, 2, 24, 15, 18, 11, 26]
Gen Order: [13, 0, 2, 24, 25, 14, 3, 10, 12, 22, 4, 26, 16, 8, 5, 1, 18, 15, 21, 17, 6, 9, 11, 19, 7, 20, 23]


No time steps to calculate are specified. I'll check the datasource of the first controller for avaiable time steps
100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 24/24 [00:01<00:00, 15.89it/s]


Iteration 5/5, Current Cost: 124.49630831133513, Best Cost: 111.55046447063715, Temperature: 773.7809374999998
Load Order: [9, 10, 16, 6, 1, 7, 22, 15, 23, 20, 21, 17, 0, 3, 12, 25, 5, 19, 4, 13, 8, 2, 24, 14, 18, 11, 26]
Gen Order: [13, 0, 2, 24, 25, 14, 3, 10, 12, 22, 4, 26, 16, 8, 19, 1, 18, 15, 21, 17, 6, 9, 11, 5, 7, 20, 23]

Total Computational Time for Simulated Annealing: 17.38 seconds

Best Total Line Loading: 110.4102826024794
Best Order Load: [9, 14, 1, 12, 16, 24, 5, 11, 17, 6, 19, 3, 10, 23, 0, 7, 20, 21, 2, 22, 4, 15, 13, 18, 8, 25, 26]
Best Order Gen: [13, 18, 20, 11, 26, 15, 9, 6, 17, 24, 25, 1, 5, 3, 14, 2, 8, 21, 4, 7, 16, 0, 22, 10, 12, 19, 23]


No time steps to calculate are specified. I'll check the datasource of the first controller for avaiable time steps
100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 24/24 [00:00<00:00, 67.71it/s]


Total Line Loading for Evaluation: 110.4102826024794


In [15]:
def refine_solution(best_load, best_gen, max_iterations=10):
    best_cost = calculate_total_line_loading(best_load, best_gen)
    tried_combinations = set()  # Keep track of index combinations that have been tried
    iteration_count = 0  # Initialize the iteration counter
    print("Initial Best Cost:", best_cost)

    for i in range(len(best_load)):
        for j in range(i + 1, len(best_load)):
            if iteration_count >= max_iterations:
                break  # Stop if maximum iterations reached
            if (i, j) in tried_combinations or (j, i) in tried_combinations:
                continue  # Skip if this combination has already been tried
            
            new_load_order = best_load[:]
            new_load_order[i], new_load_order[j] = new_load_order[j], new_load_order[i]
            new_cost = calculate_total_line_loading(new_load_order, best_gen)
            
            print(f"Iteration {iteration_count + 1}/{max_iterations}, Swapping indices {i} and {j}, Line Loading: {new_cost}, Best Cost so far: {best_cost}")
            
            if new_cost < best_cost:
                best_load = new_load_order
                best_cost = new_cost
            
            # Add the index combination to the set of tried combinations
            tried_combinations.add((i, j))
            iteration_count += 1

    return best_load, best_cost


# Further refine the best solution by swapping each index with every other index in the load order
best_load, best_cost = refine_solution(best_load, best_gen, max_iterations=10)

# Print refined best solution
print("\nFinal Refined Best Total Line Loading:", best_cost)
print("Final Refined Best Order Load:", best_load)

# Save the refined best order to CSV for evaluation
df_save = pd.DataFrame({'generation_order': best_gen, 'load_order': best_load})
df_save.to_csv(f'Group_{Group_Letter}_Assignment_2_Data_Set_C_final_refined_order.csv', index=False)

# Load again to check if it is correct
df = pd.read_csv(f'Group_{Group_Letter}_Assignment_2_Data_Set_C_final_refined_order.csv')
your_gen_order_loaded = df['generation_order'].values
your_load_order_loaded = df['load_order'].values

# Ensure the orders match
assert (list(best_gen) == list(your_gen_order_loaded)), "Generation order mismatch!"
assert (list(best_load) == list(your_load_order_loaded)), "Load order mismatch!"

# Re-create network to ensure clean state for evaluation
net = create_network()

# Evaluate the refined results
res_ext, res_lines = run_time_series(gen_data, load_data, net, index_order_load=best_load, index_order_gen=best_gen, results_suffix="evaluation_final_refined")
assert all(res_lines.max(axis=1) <= 100.0), "Line loading exceeds limit!"
print("Total Line Loading for Evaluation (Final Refined):", res_lines.max(axis=1).sum())

# Print final orders for load and generation
print("Final Generation Order:", best_gen)
print("Final Load Order:", best_load)


No time steps to calculate are specified. I'll check the datasource of the first controller for avaiable time steps
100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 24/24 [00:00<00:00, 68.67it/s]


Initial Best Cost: 110.4102826024794


No time steps to calculate are specified. I'll check the datasource of the first controller for avaiable time steps
100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 24/24 [00:00<00:00, 55.25it/s]


Iteration 1/10, Swapping indices 0 and 1, Line Loading: 110.4106871606669, Best Cost so far: 110.4102826024794


No time steps to calculate are specified. I'll check the datasource of the first controller for avaiable time steps
100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 24/24 [00:00<00:00, 53.10it/s]


...

Iteration 9/10, Swapping indices 0 and 9, Line Loading: 110.38446404915749, Best Cost so far: 108.8692307025789


No time steps to calculate are specified. I'll check the datasource of the first controller for avaiable time steps
100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 24/24 [00:01<00:00, 16.49it/s]


Iteration 10/10, Swapping indices 0 and 10, Line Loading: 108.43684505489209, Best Cost so far: 108.8692307025789

Final Refined Best Total Line Loading: 108.43684505489209
Final Refined Best Order Load: [19, 14, 9, 12, 1, 16, 5, 11, 17, 6, 24, 3, 10, 23, 0, 7, 20, 21, 2, 22, 4, 15, 13, 18, 8, 25, 26]


No time steps to calculate are specified. I'll check the datasource of the first controller for avaiable time steps
100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 24/24 [00:00<00:00, 59.97it/s]


Total Line Loading for Evaluation (Final Refined): 108.43684505489209
Final Generation Order: [13, 18, 20, 11, 26, 15, 9, 6, 17, 24, 25, 1, 5, 3, 14, 2, 8, 21, 4, 7, 16, 0, 22, 10, 12, 19, 23]
Final Load Order: [19, 14, 9, 12, 1, 16, 5, 11, 17, 6, 24, 3, 10, 23, 0, 7, 20, 21, 2, 22, 4, 15, 13, 18, 8, 25, 26]


In [16]:
def refine_generation_solution(best_load, best_gen, max_iterations=50):
    best_cost = calculate_total_line_loading(best_load, best_gen)
    tried_combinations = set()  # Keep track of index combinations that have been tried
    iteration_count = 0  # Initialize the iteration counter
    print("Initial Best Cost:", best_cost)

    for i in range(len(best_gen)):
        for j in range(i + 1, len(best_gen)):
            if iteration_count >= max_iterations:
                break  # Stop if maximum iterations reached
            if (i, j) in tried_combinations or (j, i) in tried_combinations:
                continue  # Skip if this combination has already been tried
            
            new_gen_order = best_gen[:]
            new_gen_order[i], new_gen_order[j] = new_gen_order[j], new_gen_order[i]
            new_cost = calculate_total_line_loading(best_load, new_gen_order)
            
            print(f"Iteration {iteration_count + 1}/{max_iterations}, Swapping indices {i} and {j}, Line Loading: {new_cost}, Best Cost so far: {best_cost}")
            
            if new_cost < best_cost:
                best_gen = new_gen_order
                best_cost = new_cost
            
            # Add the index combination to the set of tried combinations
            tried_combinations.add((i, j))
            iteration_count += 1

    return best_load, best_gen, best_cost

# Further refine the best generation solution by swapping each index with every other index in the generation order
best_load, best_gen, best_cost = refine_generation_solution(best_load, best_gen, max_iterations=5)

# Print refined best solution
print("\nFinal Refined Best Total Line Loading:", best_cost)
print("Final Refined Best Generation Order:", best_gen)

# Save the refined best order to CSV for evaluation
df_save = pd.DataFrame({'generation_order': best_gen, 'load_order': best_load})
df_save.to_csv(f'Group_{Group_Letter}_Assignment_2_Data_Set_C_final_refined_order.csv', index=False)

# Load again to check if it is correct
df = pd.read_csv(f'Group_{Group_Letter}_Assignment_2_Data_Set_C_final_refined_order.csv')
your_gen_order_loaded = df['generation_order'].values.tolist()
your_load_order_loaded = df['load_order'].values.tolist()

# Ensure the orders match
assert (best_gen == your_gen_order_loaded), "Generation order mismatch!"
assert (best_load == your_load_order_loaded), "Load order mismatch!"

# Re-create network to ensure clean state for evaluation
net = create_network()

# Evaluate the refined results
res_ext, res_lines = run_time_series(gen_data, load_data, net, index_order_gen=best_gen, index_order_load=best_load, results_suffix="evaluation_final_refined")
assert all(res_lines.max(axis=1) <= 100.0), "Line loading exceeds limit!"
print("Total Line Loading for Evaluation (Final Refined):", res_lines.max(axis=1).sum())

# Print final orders for load and generation
print("Final Generation Order:", best_gen)
print("Final Load Order:", best_load)



No time steps to calculate are specified. I'll check the datasource of the first controller for avaiable time steps
100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 24/24 [00:00<00:00, 62.74it/s]


Initial Best Cost: 108.43684505489209


No time steps to calculate are specified. I'll check the datasource of the first controller for avaiable time steps
100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 24/24 [00:00<00:00, 55.97it/s]


Iteration 1/5, Swapping indices 0 and 1, Line Loading: 109.14595670239163, Best Cost so far: 108.43684505489209


No time steps to calculate are specified. I'll check the datasource of the first controller for avaiable time steps
100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 24/24 [00:00<00:00, 48.31it/s]


...

Iteration 4/5, Swapping indices 0 and 4, Line Loading: 108.77591750352731, Best Cost so far: 108.43684505489209


No time steps to calculate are specified. I'll check the datasource of the first controller for avaiable time steps
100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 24/24 [00:00<00:00, 34.53it/s]


Iteration 5/5, Swapping indices 0 and 5, Line Loading: 108.4368257831481, Best Cost so far: 108.43684505489209

Final Refined Best Total Line Loading: 108.4368257831481
Final Refined Best Generation Order: [15, 18, 20, 11, 26, 13, 9, 6, 17, 24, 25, 1, 5, 3, 14, 2, 8, 21, 4, 7, 16, 0, 22, 10, 12, 19, 23]


No time steps to calculate are specified. I'll check the datasource of the first controller for avaiable time steps
100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 24/24 [00:00<00:00, 62.05it/s]


Total Line Loading for Evaluation (Final Refined): 108.4368257831481
Final Generation Order: [15, 18, 20, 11, 26, 13, 9, 6, 17, 24, 25, 1, 5, 3, 14, 2, 8, 21, 4, 7, 16, 0, 22, 10, 12, 19, 23]
Final Load Order: [19, 14, 9, 12, 1, 16, 5, 11, 17, 6, 24, 3, 10, 23, 0, 7, 20, 21, 2, 22, 4, 15, 13, 18, 8, 25, 26]
